In [2]:
import bz2
from collections import Counter
import re
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tr-abel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
!pwd

/home/tr-abel/project/gender_detector/NLPWithPytorch


In [5]:
train_file = bz2.BZ2File('amazon/Data/train.ft.txt.bz2')
test_file = bz2.BZ2File('amazon/Data/test.ft.txt.bz2')

train_file = train_file.readlines()
test_file = test_file.readlines()

In [6]:
num_train = 800000  # We're training on the first 800,000 reviews in the dataset
num_test = 200000  # Using 200,000 reviews from test set

train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_test]]

In [7]:
train_file[90]

'__label__1 No instructions included - do not trust seller: Promised with this item are "Complete Instructions" and the additional pledge that "Sweet Graces will email you with the Supply List and Instruction sheets on purchase - so you can be ready ahead of time!" I received none of this - only a plastic figurine and bracelet. To boot, Amazon claims they can do nothing to help me contact the seller. All I got was a phone number for the manufacturer. Let\'s hope that yields some results. Meanwhile, I\'m wishing I had listened to previous feedback about this unreliable seller :/\n'

In [8]:
# Extracting labels from sentences
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]

test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]

# Some simple cleaning of data
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

# Modify URLs to <url>
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [9]:
words = Counter()  # Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(train_sentences):
    # The sentences will be stored as a list of words/tokens
    train_sentences[i] = []
    for word in nltk.word_tokenize(sentence):  # Tokenizing the words
        words.update([word.lower()])  # Converting all the words to lowercase
        train_sentences[i].append(word)
    if i%20000 == 0:
        print(str((i*100)/num_train) + "% done")
print("100% done")

0.0% done
2.5% done
5.0% done
7.5% done
10.0% done
12.5% done
15.0% done
17.5% done
20.0% done
22.5% done
25.0% done
27.5% done
30.0% done
32.5% done
35.0% done
37.5% done
40.0% done
42.5% done
45.0% done
47.5% done
50.0% done
52.5% done
55.0% done
57.5% done
60.0% done
62.5% done
65.0% done
67.5% done
70.0% done
72.5% done
75.0% done
77.5% done
80.0% done
82.5% done
85.0% done
87.5% done
90.0% done
92.5% done
95.0% done
97.5% done
100% done


In [10]:
# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

In [11]:
for i, sentence in enumerate(train_sentences):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_sentences):
    # For test sentences, we have to tokenize the sentences as well
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]

In [12]:
test_labels[0:7]

[1, 1, 0, 1, 1, 0, 0]

In [13]:
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

seq_len = 200  # The length that the sentences will be padded/shortened to

train_sentences = pad_input(train_sentences, seq_len)
test_sentences = pad_input(test_sentences, seq_len)

# Converting our labels into numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [14]:
split_frac = 0.5 # 50% validation, 50% test
split_id = int(split_frac * len(test_sentences))
val_sentences, test_sentences = test_sentences[:split_id], test_sentences[split_id:]
val_labels, test_labels = test_labels[:split_id], test_labels[split_id:]

In [15]:
test_sentences[0]

array([    0,     0,     0,     0,     0,  2604,    25,    56,     9,
        4751,    13,    39,    19,   225,   217,    16,     3,  1242,
       12881,  6255,    25,    56,    78,   443,    25,    14,     3,
        1174,  4892,     4,    19,   353,    26,    34,   229,    65,
          11,  4678,    59,     9,    46,   171,     5,   370,     8,
       17591,     3,   215,    70,     4,     6,  9336,    79,    10,
           3,    60,  2517,  3416,     2,   114,     4,   175,   750,
          14,     3, 17915,   962,     4,    86,  2134,     9,    55,
          10,   189,  4702,     6,    59,     3,  4948,     8,   274,
          91,     2,    11,   147,   189,  1040,     8, 17591,   241,
          60,   244,    83,     9,   777,    50,    27,   105,    45,
        2708,     2,   235,     3,    24,  1968,     4,    86,    50,
         124,   105,    47,     7,  1841,     8,   565,     3,  3238,
          10,  9308,    21,  3019,  4129,  1025,     3,  1140,   509,
           4,     6,

### Working with Pytorch

In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

batch_size = 5000

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [17]:
test_labels[0]

1

In [18]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
# is_cuda = torch.cuda.is_available()

# # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
# if is_cuda:
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")
# device = torch.device("cpu")
device = torch.device("cuda:1")

In [19]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [20]:
len(word2idx) + 1

221523

In [21]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)

lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
epochs = 10
counter = 0
print_every = 10
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

In [22]:
# Loading the best model
model.load_state_dict(torch.load('./state_dict.pt'))

test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Test loss: 0.150
Test accuracy: 94.401%


In [29]:
len(test_loader.dataset), num_correct, len(test_losses)*5000

(100000, 94401, 100000)

#### Doing prediction on custom sentences

In [63]:
ms = "No instructions included - do not trust seller, No instructions included - do not trust seller, No instructions included - do not trust seller No instructions included - do not trust seller No instructions included - do not trust seller  No instructions included - do not trust seller No instructions included - do not trust seller No instructions included - do not trust seller: Promised with this item are \"Complete Instructions\" and the additional pledge that \"Sweet Graces will email you with the Supply List and Instruction sheets on purchase - so you can be ready ahead of time!\" I received none of this - only a plastic figurine and bracelet. To boot, Amazon claims they can do nothing to help me contact the seller. All I got was a phone number for the manufacturer. Let\'s hope that yields some results. Meanwhile, I\'m wishing I had listened to previous feedback about this unreliable seller"
msid = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(ms)]
msidp = pad_input([msid], seq_len)

In [64]:
# Loading the best model
model.load_state_dict(torch.load('./state_dict.pt'))
model.eval()

SentimentNet(
  (embedding): Embedding(221523, 400)
  (lstm): LSTM(400, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
msidp

In [ ]:
torch.from_numpy(msidp), torch.from_numpy(np.zeros(1))

In [66]:
msdata = TensorDataset(torch.from_numpy(msidp), torch.from_numpy(np.zeros(1)))
# test_labels[0]
# msdata = TensorDataset(torch.from_numpy(msidp), torch.from_numpy([test_labels[0]]))

In [67]:
msloader = DataLoader(msdata, batch_size=1)

In [68]:
h = model.init_hidden(1)

In [ ]:
for inputs, labels in msloader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    pred = torch.round(output.squeeze())  # Rounds the output to 0/1
    print(pred)

In [ ]:
output

In [ ]:
for inputs, labels in test_loader:
    